In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical

In [2]:
train_dir = './train'
correct_dir = os.path.join(train_dir, 'correct/0-correct')
not_brake_stand = os.path.join(train_dir, 'fake/1-not-on-the-brake-stand')
from_screen = os.path.join(train_dir, 'fake/2-from-the-screen')
from_screen_photoshop = os.path.join(train_dir, 'fake/3-from-the-screen+photoshop')
photoshop = os.path.join(train_dir, 'fake/4-photoshop')

In [4]:
label_categories = {
    'correct': 0,
    'not_brake_stand': 1,
    'from_screen': 2,
    'from_screen_photoshop': 3,
    'photoshop': 4
}

data = []
labels = []

def classifyImages(dir, label):
    for image_file in os.listdir(dir):
        image_path = os.path.join(dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))
        data.append(image)
        labels.append(label)


classifyImages(correct_dir, 0)
classifyImages(not_brake_stand, 1)
classifyImages(from_screen, 2)
classifyImages(from_screen_photoshop, 3)
classifyImages(photoshop, 4)

images = np.array(data)
images = images / 255.0 
labels = np.array(labels)

labels = to_categorical(labels, num_classes=5)


In [5]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

input_layer = Input(shape=(128, 128, 3)) 

# Convolutional layers
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu')(pool2)
pool3 = MaxPooling2D((2, 2))(conv3)
flatten = Flatten()(pool3)
dense1 = Dense(128, activation='relu')(flatten)
dropout = Dropout(0.5)(dense1)
output_layer = Dense(5, activation='softmax')(dropout)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856 

In [ ]:
history = model.fit(X_train, y_train, epochs=6, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/6
114/165 [===================>..........] - ETA: 15s - loss: 0.7897 - accuracy: 0.7574

In [ ]:
model.save('a5.h5')

In [ ]:
test_dir = './test'
test_data = []
for file in os.listdir(test_dir):
    img_path = os.path.join(test_dir, file)
    try:
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (128,128))
            test_data.append(img)
        else:
            print(f"Skipping {file}")
    except e:
        print(e)
        
test_data = np.array(test_data) / 255.0




In [ ]:
predict = model.predict(test_data)

In [ ]:
predicted_labels = np.argmax(predict, axis=1)


In [ ]:
for i, file in enumerate(os.listdir(test_dir)):
    print(f"File: {file}, Predicted Label: {predicted_labels[i]}")

In [ ]:
import os
import shutil
from tensorflow.keras.models import load_model

model = load_model('a5.h5')

test_dir = './test'

test_data = []

def preprocess_test_images(dir):
    for image_file in os.listdir(dir):
        image_path = os.path.join(dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))
        image = image / 255.0
        test_data.append((image_file, image))

preprocess_test_images(test_dir)

test_filenames, test_images = zip(*test_data)

test_images = np.array(test_images)

predictions = model.predict(test_images)

predicted_classes = np.argmax(predictions, axis=1)

root_dir = './root_directory3'
os.makedirs(root_dir, exist_ok=True)

for class_label in range(5): 
    class_dir = os.path.join(root_dir, f'class_{class_label}')
    os.makedirs(class_dir, exist_ok=True)

for i, file in enumerate(test_filenames):
    class_label = predicted_classes[i]
    source_path = os.path.join(test_dir, file)
    target_dir = os.path.join(root_dir, f'class_{class_label}')
    target_path = os.path.join(target_dir, file)

    shutil.copy(source_path, target_path)

print("Файлы перенесены")


In [ ]:
import csv

csv_file = 'etovse.csv'
csv_data = []

for file, predicted_label in zip(os.listdir(test_dir), predicted_labels):
    if not file.startswith('.ipynb_checkpoints'):
        if predicted_label > 1:
            predicted_label = 1
        file_name, file_extension = os.path.splitext(file)
        csv_data.append((file_name.replace('.jpeg', ''), predicted_label))

with open (csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['file_index', 'class'])
    writer.writerows(csv_data)
